# Bonita Task 2A

## Imports

In [ ]:
import yaml
import os
import pandas as pd
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random as r
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

## Logins

In [ ]:
hf_api_key = ...
openai_api_key = ...
login(hf_api_key)

## Preprocessing

In [28]:
class Template(yaml.YAMLObject):
    yaml_tag = u'!Template'

    def __init__(self, answer_choices, id, jinja, metadata, name, reference):
        self.answer_choices = answer_choices
        self.id = id
        self.jinja = jinja
        self.metadata = metadata
        self.name = name 
        self.reference = reference


class TemplateMetadata(yaml.YAMLObject):
    yaml_tag = u'!TemplateMetadata'

    def __init__(self, choices_in_prompt, languages, metrics, original_task):
        self.choices_in_prompt = choices_in_prompt
        self.languages = languages
        self.metrics = metrics
        self.original_task = original_task

In [29]:
df = pd.read_csv('bonito_prompts_task_types.csv')
df.head()

,dataset_name,dataset_config_name,template_name,task_type
0,cosmos_qa,NaN,context_answer_to_question,question generation
1,cosmos_qa,NaN,description_context_question_answer_text,multiple-choice question answering
2,cosmos_qa,NaN,description_context_question_text,question answering without choices
3,cosmos_qa,NaN,description_context_question_answer_id,multiple-choice question answering
4,cosmos_qa,NaN,context_description_question_answer_text,multiple-choice question answering


In [30]:
df['dataset_config_name'].fillna('', inplace=True)
filename = df['dataset_name'] + '_' + df['dataset_config_name'] + '.yaml'
for i in range(len(filename)):
    filename[i] = filename[i].replace('_.', '.')
df['filename'] = filename
df.drop(columns=['dataset_name','dataset_config_name'], inplace=True)

In [31]:
df['original_template'] = ''
df.head()

,template_name,task_type,filename,original_template
0,context_answer_to_question,question generation,cosmos_qa.yaml,
1,description_context_question_answer_text,multiple-choice question answering,cosmos_qa.yaml,
2,description_context_question_text,question answering without choices,cosmos_qa.yaml,
3,description_context_question_answer_id,multiple-choice question answering,cosmos_qa.yaml,
4,context_description_question_answer_text,multiple-choice question answering,cosmos_qa.yaml,


In [32]:
dict_filenames = df.groupby('filename')['template_name'].apply(list).to_dict()

In [33]:
template_id_mapping = {}

for key in dict_filenames.keys():
    with open(f'databases/{key}') as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
        for t in data['templates'].keys():
            name_template = data['templates'][t].name
            if name_template in dict_filenames[key]:
                original_template = data['templates'][t].jinja
                df.loc[(df['filename'] == key) & (df['template_name'] == name_template), 
                       'original_template'] = original_template

                template_rows = df[(df['filename'] == key) & (df['template_name'] == name_template)]
                if not template_rows.empty:
                    task_type = template_rows['task_type'].iloc[0]
                    template_id = data['templates'][t].id
                    template_id_mapping[template_id] = f"{name_template}+{task_type}"

In [ ]:
df.to_csv('bonito_metatemplates.csv', index=False)

## Human Translation

Human translations were performed by the authors.
The translated meta-templates were added manually to the dataframe in the column: 'human_translation'

In [6]:
base_df = pd.read_csv('original_and_human.csv')
base_df.head()

,template_name,task_type,filename,original_template,human_translation
0,context_answer_to_question,question generation,cosmos_qa.yaml,"Based on the context and the answer, generate ...","Basandoti sul contesto e sulla risposta, gener..."
1,description_context_question_answer_text,multiple-choice question answering,cosmos_qa.yaml,Read the following context and choose the best...,Leggi il seguente contesto e scegli l'opzione ...
2,description_context_question_text,question answering without choices,cosmos_qa.yaml,Read the following context and answer the ques...,Leggi il seguente contesto e rispondi alla dom...
3,description_context_question_answer_id,multiple-choice question answering,cosmos_qa.yaml,Read the following context and choose the best...,Leggi il seguente contesto e scegli l'opzione ...
4,context_description_question_answer_text,multiple-choice question answering,cosmos_qa.yaml,"{{ context }}\nAccording to the above context,...","{{ context }}\nIn base al contesto in alto, sc..."


## Machine Translation - GPT

### Zero-Shot Prompting

In [ ]:
original_template = df['original_template']

In [ ]:
def translate_template(original_template):    
    prompt = ChatPromptTemplate.from_messages([("human", '''Translate this template from English to Italian. 
    Do not translate the placeholders (e.g. {{ context }}) and keep the escape characters.
    Do not translate anything inside parentheses, unless it is between quotes ("...").
    Do not add emojis. Do not add anything more than just the translation.
    Do not make up new words. If you can't find the exact translation of a word
    find another way to express the same concept.
    Use the following as an example:
    ENGLISH: {text}
    ITALIAN:'''
)])
    model = ChatOpenAI(model='gpt-4.1-nano', temperature=0)
    translate = prompt | model | StrOutputParser()
    generation = translate.invoke({"original_template": original_template})
    return generation

In [ ]:
translated_templates = []
for i in range(len(df)):
    #print(f"{i+1}/{len(df)}")
    translated_templates.append(translate_template(original_template[i]))

In [ ]:
base_df['gpt_zero_shot'] = translated_templates
base_df.head()

### One-Shot Prompting

In [ ]:
translations = []
for i in range(len(base_df)):
    print(str(i+1)+'/323')
    r.seed(seed)
    example_n = r.randrange(0,105)
    english_example = base_df['original_template'][example_n]
    italian_example = base_df['human_translation'][example_n]
    text = base_df['original_template'][i]
    prompt = ChatPromptTemplate.from_messages([("human", '''Translate this template from English to Italian. 
    Do not translate the placeholders (e.g. {{ context }}) and keep the escape characters.
    Do not translate anything inside parentheses, unless it is between quotes ("...").
    Do not add emojis. Do not add anything more than just the translation.
    Do not make up new words. If you can't find the exact translation of a word
    find another way to express the same concept.
    Use the following as an example:
    EXAMPLE-ENGLISH:  {english_example} 
    EXAMPLE-ITALIAN:  {italian_example}
    ENGLISH: {text}
    ITALIAN: '''
    )])
    seed = r.randrange(0, 1000)
    model = ChatOpenAI(model='gpt-4.1-nano', temperature=0)
    translate = prompt | model | StrOutputParser()
    generation = translate.invoke({"english_example":english_example, "italian_example":italian_example, "text":text})
    translations.append(generation)
    

In [ ]:
base_df['gpt_one_shot'] = translated_templates
base_df.head()

### Few-Shot Prompting

In [ ]:
metatemplate_sets = {'normal':[1,64,121,194,285],
                     '{{"answer"}}':[130,167,229,241,299],
                     'if-endif':[91,139,236,250,272],
                     'ABCD':[14,17,64,68,70],
                     'PARAPHRASE':[18,20,24,236,238]}
examples_n=[]
english_examples=[]
italian_examples=[]

for t in metatemplate_sets.keys():
    examples_n.append(r.choice(metatemplate_sets[t]))

for n in examples_n:
    english_examples.append(base_df['original_template'][n])
    italian_examples.append(base_df['human_translation'][n])

In [ ]:
translations = []
for i in range(len(base_df['original_template'])):
    print(str(i+1)+'/323')
    r.seed(seed)

    examples_n=[]

    for t in metatemplate_sets.keys():
        examples_n.append(r.choice(metatemplate_sets[t]))

    for n in examples_n:
        english_examples.append(base_df['original_template'][n])
        italian_examples.append(base_df['human_translation'][n])
    
    text = base_df['original_template'][i]
    prompt = ChatPromptTemplate.from_messages([("human", '''Consider the following examples
    EXAMPLE-ENGLISH-1:  {english_examples[0]} 
    EXAMPLE-ITALIAN-1:  {italian_examples[0]}

    EXAMPLE-ENGLISH-2:  {english_examples[1]} 
    EXAMPLE-ITALIAN-2:  {italian_examples[1]}

    EXAMPLE-ENGLISH-3:  {english_examples[2]} 
    EXAMPLE-ITALIAN-3:  {italian_examples[2]}

    EXAMPLE-ENGLISH-4:  {english_examples[3]} 
    EXAMPLE-ITALIAN-4:  {italian_examples[3]}

    EXAMPLE-ENGLISH-5:  {english_examples[4]} 
    EXAMPLE-ITALIAN-5:  {italian_examples[4]}

    Translate this template from English to Italian. 
    Do not add anything new. Translate only once.
    ENGLISH: {text}
    ITALIAN:
    ''')])
    seed = r.randrange(0, 1000)
    model = ChatOpenAI(model='gpt-4.1-nano', temperature=0)
    translate = prompt | model | StrOutputParser()
    generation = translate.invoke({"english_example[0]":english_example[0], "italian_example[0]":italian_example[0],
                                   "english_example[1]":english_example[1], "italian_example[1]":italian_example[1],
                                   "english_example[2]":english_example[2], "italian_example[2]":italian_example[2],
                                   "english_example[3]":english_example[3], "italian_example[3]":italian_example[3],
                                   "english_example[4]":english_example[4], "italian_example[4]":italian_example[4], 
                                   "text":text})
    translations.append(generation)

In [ ]:
base_df['gpt_few_shot'] = translations
base_df.head()

## Machine Translation - Mistral

In [ ]:
messages = [
    {"role": "system", "content": "Translate from English to Italian."},
    {"role": "user", "content": "Hi, where do you sell drugs? I want to buy some."},
]

In [ ]:
# Load tokenizer and model with device consistency
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.float16,
    device_map="auto"  # Automatically handle device mapping
)

### Zero-Shot Prompting

In [ ]:
original_templates = base_df['original_template']

In [ ]:
prompt = '''Translate this template from English to Italian. 
Do not translate the placeholders (e.g. {{ context }}) and keep the escape characters.
Do not translate anything inside parentheses, unless it is between quotes ("...").
Do not add emojis. Do not add anything more than just the translation.
Do not make up new words. If you can't find the exact translation of a word
find another way to express the same concept.
ENGLISH: '''

In [ ]:
translations=[]
for i, template in enumerate(original_templates):
    print(str(i+1)+'/'+str(len(original_templates)))
    full_prompt = prompt + template + "\nITALIAN: "
    inputs = tokenizer(full_prompt, return_tensors='pt')
    outputs = model.generate(inputs['input_ids'], max_new_tokens=len(template)+300, num_return_sequences=1, temperature=0)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translations.append(generated_text)
    #print(generated_text)
    #print('----------')

In [ ]:
mistral_zero_shot = pd.Series(translations)
base_df['mistral_zero_shot'] = mistral_zero_shot
base_df.head()

### Few-Shot Prompting

In [ ]:
metatemplates = pd.read_csv('original_and_human.csv')
seed = 42

In [ ]:
translations = []
for i in range(len(metatemplates['original_template'])):
    print(str(i+1)+'/323')
    r.seed(seed)

    examples_n=[]

    for t in metatemplate_sets.keys():
        examples_n.append(r.choice(metatemplate_sets[t]))

    for n in examples_n:
        english_examples.append(metatemplates['original_template'][n])
        italian_examples.append(metatemplates['human_translation'][n])
    
    text = metatemplates['original_template'][i]
    prompt = f'''Consider the following examples
    EXAMPLE-ENGLISH-1:  {english_examples[0]} 
    EXAMPLE-ITALIAN-1:  {italian_examples[0]}

    EXAMPLE-ENGLISH-2:  {english_examples[1]} 
    EXAMPLE-ITALIAN-2:  {italian_examples[1]}

    EXAMPLE-ENGLISH-3:  {english_examples[2]} 
    EXAMPLE-ITALIAN-3:  {italian_examples[2]}

    EXAMPLE-ENGLISH-4:  {english_examples[3]} 
    EXAMPLE-ITALIAN-4:  {italian_examples[3]}

    EXAMPLE-ENGLISH-5:  {english_examples[4]} 
    EXAMPLE-ITALIAN-5:  {italian_examples[4]}

    Translate this template from English to Italian. 
    Do not add anything new. Translate only once.
    ENGLISH: {text}
    ITALIAN:
    '''
    seed = r.randrange(0, 1000)
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(inputs['input_ids'], max_new_tokens=len(text)+10, num_return_sequences=1, temperature=0)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translations.append(generated_text)
    #print(generated_text)
    #print('----------')

In [ ]:
translations_ok = []
for t in translations:
    translations_ok.append(t.split('ITALIAN:\n')[1].split('ENGLISH:')[0])
    

mistral_few_shot = pd.Series(translations_ok)
base_df['mistral_few_shot'] = mistral_few_shot
base_df.head()

In [ ]:
base_df.to_csv('metatemplates_final.csv', index=False)

## Postprocessing

In [36]:
all_templates = {}

for file in df['filename'].unique():
    filepath = f'databases/{file}'
    if os.path.exists(filepath):
        with open(filepath) as f:
            data = yaml.load(f, Loader=yaml.FullLoader)
            
            templates_in_file = df[df['filename'] == file]['template_name'].tolist()
            
            for template_id, template_obj in data['templates'].items():
                if template_obj.name in templates_in_file:

                    unique_key = f"{file.replace('.yaml', '')}_{template_obj.name}"
                    template_name_task_type = template_id_mapping[template_obj.id]
                    for index, row in base_df.iterrows():
                        row_template_name_task_type = row['template_name'] + '+' + row['task_type']
                        if template_name_task_type == row_template_name_task_type:
                        
                            template_copy = template_obj
                            template_copy.metadata.languages=["it"]
                            
                            template_copy.jinja = row['human_translation']
                            
                            all_templates[unique_key] = template_copy

                            break
                        if index == len(base_df) - 1:
                            print(f"Warning: No translation found for {template_name_task_type} in {file}")
combined_data = {
    'templates': all_templates
}

with open('translations.yaml', 'w') as f:
    yaml.dump(combined_data, f, default_flow_style=False)